In [17]:
import pandas as pd
import numpy as np
import os
import glob
import re
import math
cwd=os.getcwd()
print("Current working directory",cwd)
filenames = glob.glob(cwd + "/*.csv")
main_data=[]

for datapath in filenames:
    print("File Readind Started..........")
    print(filenames)
    dataset=pd.read_csv(datapath)
    na=["_root",'Data Crawl Completion Time(UTC)', 'Crawl Source Input', 'Brand Input']
    link=[]
    sku=[]
    for column in dataset.columns[2:len(dataset.columns)-3:2]:
        if column not in na:
            link.append(dataset[column].values)
    for column in dataset.columns[1:len(dataset.columns)-3:21]:
            sku.append(dataset[column].values)
    d=pd.DataFrame()
    d["Product Link"]=[x for x in link]
    d["root"]=dataset["_root"][0]
    d["Source"]="Flipkart"
    d["date"]=dataset["Data Crawl Completion Time(UTC)"][0]
    s=d.shape[0]
    ### ranking dictionary
    ls2=[]
    if s/24==int(s/24):
        n=int(s/24)
        for x in range(n):
            for y in range(1,25,1):
                ls2.append(y)
               
    elif s<24:
        for x in range(1,s+1,1):
            ls2.append(x)
            
    else:
        for x in range(int(s/24)):
            for y in range(1,25,1):
                ls2.append(y)
                
        for x in range(s-int(s/24)*24):
            ls2.append(x+1)
            
    d["Rank"]=[x for x in ls2]
    d["SKU"]=d["Product Link"].str.split(".com/")
    d["Product Link"]=[x[0] for x in d["Product Link"]]
    d["SKU"]=[x.split(".com/") for x in d["Product Link"]]
    sku=[]
    for x in d["SKU"]:
        sku.append(x[1])
    d["SKU"]=[x for x in sku]
    d["SKU"]=[x.split("/p/") for x in d["SKU"]]
    sku=[]
    for x in d["SKU"]:
        sku.append(x[0])
    d["SKU"]=[x for x in sku]
    ls1=["Redmi","Samsung","Poco","Realme","Oneplus","Apple","Narzo","Infinix","Iqoo","Tecno","Xiaomi","Oppo","Vivo","moto"]
    brand=[x.lower() for x in ls1]
    for i,x in enumerate(brand):
        for i,e in enumerate(d["SKU"]):
            try:
                if x in e:
                    d.loc[i,["Brand"]]=x
            except:
                print()
    
    main_data.append(d)
df=pd.concat(main_data)
df["date"]=df["date"].str.split(" ")
df["date"]=[x[0] for x in df["date"]]
brandrank=df.groupby(['date','Brand'])['Rank'].mean().reset_index()
skurank=df.groupby(['date','SKU'])['Rank'].mean().reset_index()
brandrank.columns=["date","Brand","brand rank"]
skurank.columns=["date","SKU","sku rank"]
df_brand=df.merge(brandrank,on=["date","Brand"],how="inner")
df_sku=df_brand.merge(skurank,on=["date","SKU"],how="inner")
df_sku.drop_duplicates(subset=["Brand","SKU","date"],inplace=True)
df_sku.to_csv("output.csv",index=False)

Current working directory C:\Users\HP\Documents\Xiaomi
File Readind Started..........
['C:\\Users\\HP\\Documents\\Xiaomi\\xiomi_flipkart_visibility_search_page_test_1 (8).csv', 'C:\\Users\\HP\\Documents\\Xiaomi\\xiomi_flipkart_visibility_search_page_test_1 (9).csv']
File Readind Started..........
['C:\\Users\\HP\\Documents\\Xiaomi\\xiomi_flipkart_visibility_search_page_test_1 (8).csv', 'C:\\Users\\HP\\Documents\\Xiaomi\\xiomi_flipkart_visibility_search_page_test_1 (9).csv']


In [19]:
df_sku["Brand"].unique()

array(['samsung', 'realme', 'redmi', 'apple', 'poco'], dtype=object)

In [20]:
df_sku["Brand"].nunique()

5